%md
### Let's make sure spark is working fine
Let's see what verison we're working with

In [ ]:
# spark is already initialized in zappeline
sc.version

In [ ]:
%sh
wget  http://media.sundog-soft.com/hadoop/ml-100k/u.data -O /tmp/u.data
wget  http://media.sundog-soft.com/hadoop/ml-100k/u.item -O /tmp/u.item

echo "Downloaded"

In [ ]:
%sh
hadoop fs -rm -r -f /tmp/ml-100k/
hadoop fs mkdir /tmp/ml-100k/
hadoop fs -put /tmp/u.data /tmp/ml-100k/
hadoop fs -put /tmp/u.item /tmp/ml-100k/
# SHift + Enter

# Show title -> Copy data to HDFS

In [ ]:
# scala is primary interpreter
final case class Rating(movieID: Int, rating: Int)

val lines = sc.textfile("hdfs:///tmp/ml-100k/u.data").map(x => { val fields = x.split('\t'); Rating(fields(1).toInt, fields(2).toInt  )})


In [ ]:
# convert RDD to Dataframe
import sqlContext.implicits._
val ratingsDF = lines.toDF()

ratingsDF.printSchema()


In [ ]:
val topMovieIDs = ratingsDF.groupBy('movieID').count().orderby(desc("count"))


topMovieIDs.show()

In [ ]:
# in spark 1.6
ratingsDF.registerTempTable("ratings")

In [ ]:
%sql

SELECT * FROM ratings LIMIT 10

In [ ]:
%sql

SELECT rating, count(*) AS count
FROM ratings
GROUP BY rating

clicking on visulaize will show hsitogram, pie chart 

In [ ]:
final case class Movie(movieID: Int, title: String)

val lines = sc.textfile("hdfs:///tmp/ml-100k/u.item").map(x => { val fields = x.split('|'); Movie(fields(0).toInt, fields(1)  )})

import sqlContext.implicits._
val moviesDF = lines.toDF()

moviesDF.show()

In [ ]:
moviesDF.registerTempTable("titles")

In [ ]:
%sql

SELECT t.title, count(*) as cnt
FROM ratinsg r
JOIN titles t
ON r.movieID= t.movieID
GROUP BY t.title
ORDER BY cnt DESC LIMIT 20

In [ ]:
# long tail